In [52]:
import warnings
warnings.filterwarnings('ignore')

In [53]:
import pandas as pd
import numpy as np
import spacy
# import nltk
# nltk.download('punkt')

In [54]:
cd converted_to_dataframe

[WinError 2] The system cannot find the file specified: 'converted_to_dataframe'
D:\FSOFT_QAI\Mock project\PRE_RUNNING\TANDZ\converted_to_dataframe


## Analysis on train_set

#### TH1: Nhãn PERSON, PERTYPE ở label1

In [55]:
train_df = pd.read_csv('train-converted.csv')
train_df['label1'].value_counts()

O                      550851
PERSON                   7897
ORGANIZATION             5173
LOCATION-GPE             4986
LOCATION                 3929
QUANTITY-NUM             2922
PERSONTYPE               2634
QUANTITY                 2003
DATETIME                 1468
PRODUCT                  1376
ORGANIZATION-SPORTS      1358
DATETIME-DATE             960
DATETIME-DURATION         653
QUANTITY-CUR              626
QUANTITY-ORD              535
PRODUCT-COM               521
MISCELLANEOUS             465
LOCATION-STRUC            405
EVENT-SPORT               400
EVENT                     337
QUANTITY-PER              309
EVENT-GAMESHOW            294
DATETIME-TIME             273
LOCATION-GEO              265
QUANTITY-DIM              262
QUANTITY-AGE              260
DATETIME-TIMERANGE        218
DATETIME-DATERANGE        210
URL                       165
EVENT-CUL                 162
PHONENUMBER               144
ORGANIZATION-MED          133
PRODUCT-LEGAL             117
EVENT-NATU

In [56]:
PER_01 = train_df[train_df['label1']=='PERSON']
PERTYPE_01 = train_df[train_df['label1']=='PERSONTYPE']

In [57]:
total = train_df['label1'].value_counts().sum()
num_PER_01 = PER_01.value_counts().sum()
num_PERTYPE_01 = PERTYPE_01.value_counts().sum()

rate_PER_01 = num_PER_01 / total
rate_PERTYPE_01 = num_PERTYPE_01 / total

In [58]:
print(f'Số lượng nhãn PERSON: {num_PER_01} chiếm {rate_PER_01 * 100:.4f}% trên tổng số {total} nhãn. \
        \nSố lượng nhãn PERSONTYPE: {num_PERTYPE_01} chiếm {rate_PERTYPE_01 * 100:.4f}% trên tổng số {total} nhãn.')

Số lượng nhãn PERSON: 7897 chiếm 1.3324% trên tổng số 592683 nhãn.         
Số lượng nhãn PERSONTYPE: 2634 chiếm 0.4444% trên tổng số 592683 nhãn.


In [59]:
PER_NESTED_01 = PER_01[(PER_01['label2']!='?') & (PER_01['label2']!='O')]
PER_NESTED_01

,f_name,n_sent,word,label1,label2,label3


==> Đối với nhãn PERSON ở label1, dữ liệu **không có nhãn lồng.**

In [60]:
PERTYPE_NESTED_01 = PERTYPE_01[(PERTYPE_01['label2']!='?') & (PERTYPE_01['label2']!='O')]
PERTYPE_NESTED_01['label2'].value_counts()

I-ORGANIZATION        34
I-LOCATION-GPE        31
I-LOCATION             4
I-ORGANIZATION-MED     3
B-LOCATION-GPE         3
I-LOCATION-GEO         1
I-LOCATION-STRUC       1
B-ORGANIZATION         1
Name: label2, dtype: int64

==> Ở đây chúng ta có thể thấy PERTYPE bị **nhãn lồng** chủ yếu ở ORG và LOC, điều này cũng khá dễ hiểu vì chức vụ thì thường đi cùng tổ chức, hoặc quận, huyện, TP, quốc gia.
- Tỷ lệ **nhãn lồng** ở PERTYPE chiếm: 2.9613% trong tổng số nhãn PERTYPE
- Trong description, người ta chỉ mô tả ORG-MED, ORG-STOCK, ORG-SPORT. Vậy thì những ORG còn lại khác chúng ta tự hiểu là ORG chung hay có thể cân nhắc gán thành LOC-GPE hay không vì 2 nhãn này khá nhập nhằng và giống nhau?

In [61]:
PERTYPE_NESTED_01[PERTYPE_NESTED_01['label2']=='I-ORGANIZATION'].head(10)

,f_name,n_sent,word,label1,label2,label3
15006,xahoi_0026,22,Chính ủy BTL Thủ đô,PERSONTYPE,I-ORGANIZATION,?
15039,xahoi_0026,70,Phó tư lệnh BTL Thủ đô,PERSONTYPE,I-ORGANIZATION,?
54648,xahoi_0045,15,Thứ trưởng Bộ Công an,PERSONTYPE,I-ORGANIZATION,?
64283,xahoi_0024,22,Uỷ viên Bộ Chính trị,PERSONTYPE,I-ORGANIZATION,?
64287,xahoi_0024,34,Chủ nhiệm UBKT Trung ương,PERSONTYPE,I-ORGANIZATION,?
125453,xahoi_0083,49,Phó cục trưởng Cục CSGT,PERSONTYPE,I-ORGANIZATION,?
136737,xahoi_0022,13,Tổng cục trưởng Tổng cục Đường bộ,PERSONTYPE,I-ORGANIZATION,?
182456,khcn_0068,15,trưởng Viện Nghiên cứu Phát triển,PERSONTYPE,I-ORGANIZATION,?
182492,khcn_0068,12,trưởng Viện Viện NCPT Du lịch,PERSONTYPE,I-ORGANIZATION,?
270507,xahoi_0021,11,"Tổng Cục trưởng Tổng cục Phòng , Chống thiên tai",PERSONTYPE,I-ORGANIZATION,?


**Ý kiến cá nhân:** đối với PERTYPE lồng ORG thì nên gán tất cả là PERTYPE

In [62]:
PERTYPE_NESTED_01[PERTYPE_NESTED_01['label2']=='I-LOCATION-GPE'].head(10)

,f_name,n_sent,word,label1,label2,label3
14965,xahoi_0026,22,Phó Giám đốc CA TP Hà Nội,PERSONTYPE,I-LOCATION-GPE,?
25206,xahoi_0047,5,Phó chủ tịch UBND huyện Tuy Phong,PERSONTYPE,I-LOCATION-GPE,?
64726,xahoi_0024,3,Chủ tịch HĐND TP Đà Nẵng,PERSONTYPE,I-LOCATION-GPE,?
64803,xahoi_0024,18,Chủ tịch UBND TP Đà Nẵng,PERSONTYPE,I-LOCATION-GPE,?
162524,xahoi_0084,7,Đại sứ Vương quốc Thụy Điển,PERSONTYPE,I-LOCATION-GPE,?
203558,khcn_0071,3,người Ai Cập,PERSONTYPE,I-LOCATION-GPE,?
203685,khcn_0071,22,người Ai Cập,PERSONTYPE,I-LOCATION-GPE,?
207669,xahoi_0046,10,Chủ tịch UBND quận Hoàng Mai,PERSONTYPE,I-LOCATION-GPE,?
207759,xahoi_0046,10,Chủ tịch UBND quận Hoàng Mai,PERSONTYPE,I-LOCATION-GPE,?
270514,xahoi_0021,32,Phó Chủ tịch UBND tỉnh Nghệ An,PERSONTYPE,I-LOCATION-GPE,?


**Ý kiến cá nhân:** đối với PERTYPE lồng LOC-GPE thì nên gán là PERTYPE-LOC
- người Ai Cập: MISC or PERTYPE? Anh đùa tôi à anh data :))

In [63]:
list_sen_PER_01 = PER_01['f_name'].unique().tolist()
list_sen_PERTYPE_01 = PERTYPE_01['f_name'].unique().tolist()

In [64]:
import spacy
from spacy import displacy

colors = ['Apricot', 'Brown', 'Olive', 'Teal', 'Pink', 'Black', 'Red', 'Orange', 'Yellow', 'Lime', 'Green', 'Cyan', 'Blue', 'Purple', 
         'Grey']
NER = ['DATETIME', 'PERSONTYPE', 'PERSON', 'ORGANIZATION', 'PRODUCT', 'EVENT', 'LOCATION', 'URL', 'PHONENUMBER', 'QUANTITY', 'IP', 'ADDRESS', 'SKILL', 'EMAIL', 'MISCELLANEOUS']
COLORS = dict()
for i in range(len(NER)):
    COLORS[NER[i]] = colors[i]
OPTIONS = {'ents': NER, 'colors': COLORS}
    
## visualize result
## input: predict format [(word, tag)]

def visualize_spacy(arr):
    if len(arr) < 1:
        return None
    text = ' '.join([i for i, j in arr])
    pos = 0
    start_end_labels = []
    for word, tag in arr:
        if len(start_end_labels) > 0 and tag == start_end_labels[-1][2]:
            temp = [start_end_labels[-1][0], pos+len(word), tag]
            start_end_labels[-1] = temp.copy()
        else:
            temp = [pos, pos+len(word), tag]
            start_end_labels.append(temp)
        pos += len(word) + 1
        
    ex = [{'text': text, 'ents': [{'start': x[0], 'end': x[1], 'label': x[2]} for x in start_end_labels if x[2]!= 0]}]
    displacy.render(ex, manual=True, jupyter=True, style='ent', options = OPTIONS )

In [77]:
from tkinter import *
from IPython.display import clear_output
root = Tk()
root.geometry("240x100")
root.title("GUI Visualize NER")

global counter
counter = -1

def nClick(args):
    global counter
    counter += 1
    mButton1.config(text = 'Sentence ' + str(counter))
    if args == 'PERSON':
        file_name = list_sen_PER_01[counter]
    if args == 'PERSONTYPE':
        file_name = list_sen_PERTYPE_01[counter]
        
    input_visual_word = train_df[train_df['f_name']==file_name]['word'].values.tolist()
    input_visual_tag = train_df[train_df['f_name']==file_name]['label1'].values.tolist()
    input_visual = [(word, tag) for word, tag in zip(input_visual_word, input_visual_tag)]
    visualize_spacy(input_visual)
    clear_output(wait=True)


#mButton1 = Button(text = "Begin Visualize", command = nClick, fg = "darkgreen", bg = "white")
mButton1 = Button(text = "Begin Visualize", command = lambda: nClick('PERSONTYPE'))
mButton1.pack()

root.mainloop()